In [1]:
import csv
import json
import os
import pprint
from google.colab import drive

# Mount Google Drive (if using Google Colab)
drive.mount('/content/drive')

def load_data():
    csv_file_path = '/content/sample_data/sales_data.csv'
    json_folder_path = '/content/sample_data/details'
    txt_folder_path = '/content/sample_data/discription'

    sales_data = {}
    try:
        with open(csv_file_path, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    sales_data[row['Product_SKU']] = {
                        'Product_SKU': row['Product_SKU'],
                        'Sales_Data': json.loads(row['Sales_Data'])
                    }
                except KeyError as e:
                    print(f"Missing key in CSV row: {e}")
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON for SKU {row['Product_SKU']}: {e}")
    except FileNotFoundError:
        print(f"CSV file not found: {csv_file_path}")

    product_details = {}
    for filename in os.listdir(json_folder_path):
        if filename.endswith('.json'):
            try:
                with open(os.path.join(json_folder_path, filename), 'r') as file:
                    data = json.load(file)
                    product_details[filename] = data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON file {filename}: {e}")
            except FileNotFoundError:
                print(f"JSON file not found: {filename}")

    product_descriptions = {}
    for filename in os.listdir(txt_folder_path):
        if filename.endswith('.txt'):
            try:
                with open(os.path.join(txt_folder_path, filename), 'r') as file:
                    description = file.read()
                    product_descriptions[filename] = description
            except FileNotFoundError:
                print(f"TXT file not found: {filename}")

    return sales_data, product_details, product_descriptions

def add_sales_data(sales_data, sku, sales_data_values):
    sales_data[sku] = {
        'Product_SKU': sku,
        'Sales_Data': sales_data_values
    }

    csv_file_path = '/content/sample_data/sales_data.csv'
    file_exists = os.path.isfile(csv_file_path)

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['Product_SKU', 'Sales_Data'])
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            'Product_SKU': sku,
            'Sales_Data': json.dumps(sales_data_values)
        })

def add_product_details(product_details, sku, details):
    product_detail_filename = f'{sku}.json'
    product_details[product_detail_filename] = details

    json_file_path = os.path.join('/content/sample_data/details', product_detail_filename)
    with open(json_file_path, 'w') as file:
        json.dump(details, file, indent=4)

def add_product_description(product_descriptions, sku, description):
    product_description_filename = f'{sku}.txt'
    product_descriptions[product_description_filename] = description

    txt_file_path = os.path.join('/content/sample_data/discription', product_description_filename)
    with open(txt_file_path, 'w') as file:
        file.write(description)

def create():
    sku = 'TYS56KFJW93NJ'
    sales_data_values = [0] * 14
    product_details = {
        'Product name': 'Laptop',
        'Brand': 'Acer',
        'Model': 'Acer Aspire 3',
        'Specifications': 'Intel Core i3 N305 Laptop (Windows 11 Home/8 GB/512 GB SSD) A314-36M, 35.56 cm (14") Full HD Display, 1.4 KG, Pure Silver',
        'Price': 'INR 32,999.00',
        'Availability': 'In stock'
    }
    product_description = 'The Aspire 3 is ready to go with the latest Intel® Core™ i3 N-Series Processors1 with UHD Graphics—ideal for the entire family, with performance and productivity at the core. Perfect to get more out of work, study, or play.'

    sales_data, product_details_dict, product_descriptions_dict = load_data()

    add_sales_data(sales_data, sku, sales_data_values)
    add_product_details(product_details_dict, sku, product_details)
    add_product_description(product_descriptions_dict, sku, product_description)

def display_sales_data(sales_data, sku):
    if sku in sales_data:
        print(f"Sales Data for SKU {sku}:")
        pprint.pprint(sales_data[sku]['Sales_Data'])
    else:
        print(f"No sales data found for SKU {sku}")

def display_product_details(product_details, sku):
    product_detail_filename = f'{sku}.json'
    if product_detail_filename in product_details:
        print(f"Product Details for SKU {sku}:")
        pprint.pprint(product_details[product_detail_filename])
    else:
        print(f"No product details found for SKU {sku}")

def display_product_description(product_descriptions, sku):
    product_description_filename = f'{sku}.txt'
    if product_description_filename in product_descriptions:
        print(f"Product Description for SKU {sku}:")
        print(product_descriptions[product_description_filename])
    else:
        print(f"No product description found for SKU {sku}")

def read(sales_data, product_details, product_descriptions, sku):
    display_sales_data(sales_data, sku)
    display_product_details(product_details, sku)
    display_product_description(product_descriptions, sku)

# Call the create function to add the new product
create()

# Reload and display updated data
sales_data, product_details, product_descriptions = load_data()

# Display all information for Acer Aspire 3
sku = 'TYS56KFJW93NJ'
read(sales_data, product_details, product_descriptions, sku)
def update_sales_data(sales_data, sku, new_sales_data_values):
    # Update the sales data for the given SKU
    if sku in sales_data:
        sales_data[sku]['Sales_Data'] = new_sales_data_values
        print(f"Updated sales data for SKU {sku}")

        # Write the updated data to the CSV file
        csv_file_path = '/content/sample_data/sales_data.csv'
        temp_file_path = '/content/sample_data/sales_data_temp.csv'

        # Read existing CSV data
        with open(csv_file_path, mode='r') as infile, open(temp_file_path, mode='w', newline='') as outfile:
            reader = csv.DictReader(infile)
            writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)

            writer.writeheader()
            for row in reader:
                if row['Product_SKU'] == sku:
                    row['Sales_Data'] = json.dumps(new_sales_data_values)
                writer.writerow(row)

        # Replace the old CSV file with the updated one
        os.replace(temp_file_path, csv_file_path)
    else:
        print(f"No sales data found for SKU {sku}")
def update_product_details(product_details, sku, new_details):
    product_detail_filename = f'{sku}.json'
    if product_detail_filename in product_details:
        product_details[product_detail_filename] = new_details
        print(f"Updated product details for SKU {sku}")

        # Write the updated data to the JSON file
        json_file_path = os.path.join('/content/sample_data/details', product_detail_filename)
        with open(json_file_path, 'w') as file:
            json.dump(new_details, file, indent=4)
    else:
        print(f"No product details found for SKU {sku}")
def update_product_description(product_descriptions, sku, new_description):
    product_description_filename = f'{sku}.txt'
    if product_description_filename in product_descriptions:
        product_descriptions[product_description_filename] = new_description
        print(f"Updated product description for SKU {sku}")

        # Write the updated description to the TXT file
        txt_file_path = os.path.join('/content/sample_data/discription', product_description_filename)
        with open(txt_file_path, 'w') as file:
            file.write(new_description)
    else:
        print(f"No product description found for SKU {sku}")
def update(sales_data, product_details, product_descriptions, sku, new_sales_data_values, new_details, new_description):
    update_sales_data(sales_data, sku, new_sales_data_values)
    update_product_details(product_details, sku, new_details)
    update_product_description(product_descriptions, sku, new_description)
# Load existing data
sales_data, product_details, product_descriptions = load_data()

# Define new data to update
sku = 'TYS56KFJW93NJ'
new_sales_data_values = [100] * 14  # Example: updated sales data
new_details = {
    'Product name': 'Laptop',
    'Brand': 'Acer',
    'Model': 'Acer Aspire 3',
    'Specifications': 'Updated specifications here',
    'Price': 'INR 34,999.00',  # Updated price
    'Availability': 'In stock'
}
new_description = 'Updated description here.'

# Call the update function
update(sales_data, product_details, product_descriptions, sku, new_sales_data_values, new_details, new_description)

# Reload and display updated data to verify
sales_data, product_details, product_descriptions = load_data()
read(sales_data, product_details, product_descriptions, sku)


def delete(sales_data, product_details, product_descriptions, sku):
    # Remove sales data
    if sku in sales_data:
        del sales_data[sku]
        print(f"Removed sales data for SKU {sku}")

        # Write the updated data to the CSV file
        csv_file_path = '/content/sample_data/sales_data.csv'
        temp_file_path = '/content/sample_data/sales_data_temp.csv'

        # Read existing CSV data
        with open(csv_file_path, mode='r') as infile, open(temp_file_path, mode='w', newline='') as outfile:
            reader = csv.DictReader(infile)
            writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
            writer.writeheader()
            for row in reader:
                if row['Product_SKU'] != sku:
                    writer.writerow(row)

        # Replace the old CSV file with the updated one
        os.replace(temp_file_path, csv_file_path)
    else:
        print(f"No sales data found for SKU {sku}")

    # Remove product details
    product_detail_filename = f'{sku}.json'
    if product_detail_filename in product_details:
        del product_details[product_detail_filename]
        print(f"Removed product details for SKU {sku}")

        json_file_path = os.path.join('/content/sample_data/details', product_detail_filename)
        if os.path.isfile(json_file_path):
            os.remove(json_file_path)
        else:
            print(f"JSON file not found: {json_file_path}")
    else:
        print(f"No product details found for SKU {sku}")

    # Remove product description
    product_description_filename = f'{sku}.txt'
    if product_description_filename in product_descriptions:
        del product_descriptions[product_description_filename]
        print(f"Removed product description for SKU {sku}")

        txt_file_path = os.path.join('/content/sample_data/discription', product_description_filename)
        if os.path.isfile(txt_file_path):
            os.remove(txt_file_path)
        else:
            print(f"TXT file not found: {txt_file_path}")
    else:
        print(f"No product description found for SKU {sku}")

# Load existing data
sales_data, product_details, product_descriptions = load_data()

# Define SKU to delete
sku = 'TYS56KFJW93NJ'

# Call the delete function
delete(sales_data, product_details, product_descriptions, sku)

# Verify removal by reloading data
sales_data, product_details, product_descriptions = load_data()
read(sales_data, product_details, product_descriptions, sku)



Mounted at /content/drive
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
Missing key in CSV row: 'Sales_Data'
No sales data found for SKU TYS56KFJW93NJ
Product Details for SKU TYS56KFJW93NJ:
{'Availability': 'In stock',
 'Brand': 'Acer',
 'Model': 'Acer Aspire 3',
 'Price': 'INR 32,999.00',
 'Product name'